# HW 9: Web Scraping with Selectors
Submitted by: Gideon Tay\
My UNI: gt2528\
Contact me at: gideon.tay@columbia.edu

## 1. Create a Function to Scrape Company Infoboxes
### 1a. Write a function called `get_company_info`

Let us first import the necessary libaries and create a function that normalizes financial data to billions of USD:

In [38]:
# Import necessary libraries
import requests
from lxml import html
import re
import warnings

# Function to normalize financial numbers to billions
def to_billions(value):
    # Save the original string for unit detection
    original_value = value.lower()
    
    # Make value lowercase
    value = value.lower()
    # Remove years in parentheses eg. "(2024)", "(Feb 2024)"
    value = re.sub(r'\(\d{4}\)|\([A-Za-z]{3} \d{4}\)', '', value)
    # Remove commas, $ and US$
    value = re.sub(r'[,$]|US\$', '', value)

    # Extract the first sequence of numbers with optional decimal point
    # While we already accounted for cases where multi year data is given
    # in <ul><li> format to take the first item only, there are some pages with
    # multi-year data in <span>. Just take the most recent year's (first sequence's)
    # data if that is the case
    match = re.search(r'\d+(\.\d+)?', value)
    if match:
        value = float(match.group())
    else:
        warnings.warn(f"Unable to find financial value in '{original_value}'")

    # Ensure the value is in USD. Convert to USD if in EUR. 
    usd_keywords = ['usd', 'us$', 'united states dollar', '$']
    if not any(keyword in original_value for keyword in usd_keywords):
        if '€'in original_value.lower():
            value = value * 1.0651 # Convert from EUR to USD exchange rate
        else: # If not USD or EUR (to convert), return None
            warnings.warn(f"Value '{original_value}' is not in USD")
            return None

    # Determine the scale based on the original value text
    if 'billion' in original_value:
        return value  # Already in billions
    elif 'million' in original_value:
        return value / 1000  # Convert million to billion
    elif 'thousand' in original_value:
        return value / 1000000 # Convert thousand to billion
    else: # no units given so it is in plain USD
        return value / 1000000000 # Convert from USD to billion

Let us also create a function to normalize employee data to level units:

In [65]:
def norm_employee(value):
    # Save the original string for unit detection
    original_value = value.lower()

    # Remove any paranthesis if there is a 4 digit no. in it
    # Removes (2023), (3 Dec 2023), (Sept. 2024), (Jan 31, 2024)
    value = re.sub(r'\([^\)]*\d{4}[^\)]*\)', '', value)
    # Remove truncated words "c.", "approx.", "Sept."
    value = re.sub(r'\b[a-zA-Z]+\.', '', value)
    # Remove .m and .t, seen in some wikipedia webpage for style
    value = re.sub(r'.m|.t', '', value)
    # Remove any non-numeric characters except for the decimal point
    value = re.sub(r'[^\d\.]', '', value) 

    # Determine the scale based on the original value text
    if 'billion' in original_value:
       return value * 1000000000 # Convert billion to level
    elif 'million' in original_value:
        return value * 1000000  # Convert million to level
    elif 'thousand' in original_value:
        return value * 1000 # Convert thousand to level
    else: # no units given so it is already level
       return value

Now, let's write the `get_company_info` function:

In [40]:
def get_company_info(url: str):
    # Fetch the HTML content
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    tree = html.fromstring(response.content) # Parse the HTML content

    # Select the infobox using its class
    # Try to find the infobox using either of the two possible CSS selectors
    css1 = ".infobox.ib-company.vcard"
    css2 = ".infobox.vcard"
    infobox = tree.cssselect(css1) or tree.cssselect(css2)
    if not infobox:
        raise RuntimeError("Infobox not found on the Wikipedia page")
    infobox = infobox[0] # Convert this from list to element

    # Setup empty dictionary
    data = {
        "Company Name": None,
        "Industry": None,
        "Revenue (Billions USD)": None,
        "Net Income (Billions USD)": None,
        "Number of Employees": None,
        "Market Cap (Billions USD)": None
    }

    # Get the company name text from the title element in infobox
    name_element = infobox.cssselect('.infobox-title.fn.org')
    if name_element:
        data["Company Name"] = name_element[0].text_content().strip()

    # Loop through the rows in the infobox
    for row in infobox.cssselect('tr'):
        label = row.cssselect('th.infobox-label') 
        value = row.cssselect('td.infobox-data')
        # Skip cases of <tr> without the label and value structure
        if not label or not value:
            continue

        label_text = label[0].text_content().strip().lower()
        value_element = value[0]

        # Handle unordered lists or multiple elements in the value field
        multi_value_col = ['industry', 'sector']
        if value_element.cssselect('ul li'):
            # For industry/ founders/ key people, take all list elements
            if any(col in label_text for col in multi_value_col):
                value_text = ', '.join(
                    li.text_content().strip() for li in value_element.cssselect('ul li')
                    )
            else: # For other columns, take first element in list
                value_text = value_element.cssselect('ul li')[0].text_content().strip()
        # If no unordered list, just return text content
        else:
            value_text = value_element.text_content().strip()

        # Replace non-breaking spaces in the value text
        value_text = value_text.replace('\xa0', ' ')
        # Remove citations in Wikipedia like [3] and [12]
        value_text = re.sub(r'\[\d{1,2}\]', '', value_text)

        # Allow some variability in field names eg. industry or sector
        if 'industry' in label_text or 'sector' in label_text:
            data["Industry"] = value_text
        elif 'revenue' in label_text:
            data["Revenue (Billions USD)"] = to_billions(value_text)
        elif 'net income' in label_text or 'net profit' in label_text:
            data["Net Income (Billions USD)"] = to_billions(value_text)
        elif 'employees' in label_text:
            data["Number of Employees"] = norm_employee(value_text)
        elif 'market cap' in label_text or 'market capitalization' in label_text:
            data["Market Cap (Billions USD)"] = value_text
    
    return data

Now, let's test the `get_company_info` function with the Apple Inc. page to verify that it works:

In [41]:
# Test with Apple Inc. Wikipedia page
get_company_info('https://en.wikipedia.org/wiki/Apple_Inc.')

{'Company Name': 'Apple Inc.',
 'Industry': 'Consumer electronics, Software services, Online services',
 'Revenue (Billions USD)': 391.04,
 'Net Income (Billions USD)': 93.74,
 'Number of Employees': '164000',
 'Market Cap (Billions USD)': None}

### 1b. Expand the `get_company_info` function
Now, scrape information on key people (CEO, founders, and founded year) as well. Let's call the expanded function `get_company_info2`:

In [ ]:
def get_company_info2(url: str):
    # Fetch the HTML content
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    tree = html.fromstring(response.content) # Parse the HTML content

    # Select the infobox using its class
    # Try to find the infobox using either of the two possible CSS selectors
    css1 = ".infobox.ib-company.vcard"
    css2 = ".infobox.vcard"
    infobox = tree.cssselect(css1) or tree.cssselect(css2)
    if not infobox:
        raise RuntimeError("Infobox not found on the Wikipedia page")
    infobox = infobox[0] # Convert this from list to element

    # Setup empty dictionary
    data = {
        "Company Name": None,
        "Industry": None,
        "Revenue (Billions USD)": None,
        "Net Income (Billions USD)": None,
        "Number of Employees": None,
        "Market Cap (Billions USD)": None,
        "CEO": None,
        "Founder(s)": None,
        "Founded (year)": None
    }

    # Get the company name text from the title element in infobox
    name_element = infobox.cssselect('.infobox-title.fn.org')
    if name_element:
        data["Company Name"] = name_element[0].text_content().strip()

    # Loop through the rows in the infobox
    for row in infobox.cssselect('tr'):
        label = row.cssselect('th.infobox-label') 
        value = row.cssselect('td.infobox-data')
        # Skip cases of <tr> without the label and value structure
        if not label or not value:
            continue

        label_text = label[0].text_content().strip().lower()
        value_element = value[0]

        # Handle unordered lists or multiple elements in the value field
        multi_value_col = ['industry', 'sector', 'key people', 'founder', 'founding team']
        if value_element.cssselect('ul li'):
            # For industry/ founders/ key people, take all list elements
            if any(col in label_text for col in multi_value_col):
                value_text = ', '.join(
                    li.text_content().strip() for li in value_element.cssselect('ul li')
                    )
            else: # For other columns, take first element in list
                value_text = value_element.cssselect('ul li')[0].text_content().strip()
        # If no unordered list, just return text content
        else:
            value_text = value_element.text_content().strip()

        # Replace non-breaking spaces in the value text
        value_text = value_text.replace('\xa0', ' ')
        # Remove citations in Wikipedia like [3] and [12]
        value_text = re.sub(r'\[\d{1,2}\]', '', value_text)

        # Allow some variability in field names eg. industry or sector
        # Note perfect matching is not needed, as long as the keyword is in lable_text
        if 'industry' in label_text or 'sector' in label_text:
            data["Industry"] = value_text
        elif 'revenue' in label_text:
            data["Revenue (Billions USD)"] = to_billions(value_text)
        elif 'net income' in label_text or 'net profit' in label_text:
            data["Net Income (Billions USD)"] = to_billions(value_text)
        elif 'employees' in label_text: # Matches 'no. of employees', 'number of employees' etc. 
            data["Number of Employees"] = norm_employee(value_text)
        elif 'market cap' in label_text: # Matches 'market cap', ''market capitalization' etc.
            data["Market Cap (Billions USD)"] = value_text
        # Extract CEO field if present (unlikely, usually under key people)
        elif 'ceo' in label_text:
            data["CEO"] = value_text
        # Extract founder from founder field
        elif 'founder' in label_text or 'founding team' in label_text: # Matches 'founder', 'founders', 'founding_team' etc.
            data["Founder(s)"] = value_text
        # Extract founded year from founded field
        elif 'founded' in label_text:
            pattern = r'\b(17|18|19|20)\d{2}\b' # Regex pattern to extract year
            match = re.search(pattern, value_text) # Extract year from text
            if match: # if no match, then don't update dict
                data["Founded (year)"] = match.group()
        # Extract CEO from key people eg. Tim Cook (CEO)
        elif 'key people' in label_text:
            # Regex for CEO, allow period in name for abbreviations like John F. Kennedy
            # Optional comma after the name and make () optional for cases like "John, CEO"
            # Allow for multiple roles like (chairman & CEO) with .*
            ceo_pattern = r'\b([\w\s\.]+),?\s*\(?.*(?:Chief Executive Officer|CEO).*\)?' 
            ceo_match = re.search(ceo_pattern, value_text)
            if ceo_match: # if no match, then don't update dict
                data["CEO"] = ceo_match.group(1).strip()

    return data

# Test with Apple Inc. Wikipedia page
get_company_info2('https://en.wikipedia.org/wiki/Apple_Inc.')

{'Company Name': 'Apple Inc.',
 'Industry': 'Consumer electronics, Software services, Online services',
 'Revenue (Billions USD)': 391.04,
 'Net Income (Billions USD)': 93.74,
 'Number of Employees': '164000',
 'Market Cap (Billions USD)': None,
 'CEO': 'Arthur Levinson',
 'Founder(s)': 'Steve Jobs, Steve Wozniak, Ronald Wayne',
 'Founded (year)': '1976'}

## 2. Retrieve S&P 500 Table
Let's scrape the S&P 500 Table from the [List of S&P 500 companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies) Wikipedia page and create a pandas dataframe:

In [76]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the S&P 500 Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Send a GET request
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first table on the page, S&P500 Constituents is the first table
table = soup.find('table', {'class': 'wikitable'})

# Extract table rows, contained in <tr> HTML tags
rows = table.find_all('tr')

# Extract table data
data = []
for row in rows[1:]: # Skip first row of titles
    cols = row.find_all('td') # Each row's cell in <td> tags
    # Hard code in which info is in which col
    # No other method of identification since there is no class/ id in <td> tag
    if cols: 
        company_name = cols[1].text.strip()
        link = cols[1].find('a')['href']
        gics_sector = cols[2].text.strip()
        gics_sub_industry = cols[3].text.strip()
        headquarters = cols[4].text.strip()
        data.append([
            company_name, "https://en.wikipedia.org" + link, 
            gics_sector, gics_sub_industry, headquarters
            ])

# Create a DataFrame
df = pd.DataFrame(data, columns=[
    "Company Name", "Link to Company Wikipedia page", 
    "GICS Sector", "GICS Sub-Industry", "Headquarters Location"
    ])

# Display the first five rows
df.head()

,Company Name,Link to Company Wikipedia page,GICS Sector,GICS Sub-Industry,Headquarters Location
0,3M,https://en.wikipedia.org/wiki/3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota"
1,A. O. Smith,https://en.wikipedia.org/wiki/A._O._Smith,Industrials,Building Products,"Milwaukee, Wisconsin"
2,Abbott Laboratories,https://en.wikipedia.org/wiki/Abbott_Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois"
3,AbbVie,https://en.wikipedia.org/wiki/AbbVie,Health Care,Biotechnology,"North Chicago, Illinois"
4,Accenture,https://en.wikipedia.org/wiki/Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland"


## 3. Add Detailed Information to the S&P 500 DataFrame
 
Use the `get_company_info2` function created for question 1b to retrieve additional information for each company in the S&P 500 dataframe created in question 2.

In [84]:
# Initialize new columns for additional info
new_columns = [
    "Industry", "Revenue (Billions USD)", 
    "Net Income (Billions USD)", "Number of Employees", 
    "Market Cap (Billions USD)", "CEO", 
    "Founder(s)", "Founded (year)"
    ]
for col in new_columns:
    df[col] = None # set content in columns as None for now

# Loop through each company and enrich the DataFrame
for i, row in df.iterrows():
    try:
        url = row['Link to Company Wikipedia page']
        company_info = get_company_info2(url)
        
        # Update the DataFrame with the retrieved data
        for col in new_columns:
            df.at[i, col] = company_info.get(col)
    
    except Exception as e:
        print(f"Failed to retrieve data for {row['Company Name']}: {e}")

# Display the first few rows of the enriched DataFrame
df.head()

C:\Users\gideo\AppData\Local\Temp\ipykernel_5048\2840923209.py:36: UserWarning: Value '1.123 billion (2023)' is not in USD
  warnings.warn(f"Value '{original_value}' is not in USD")


Failed to retrieve data for Insulet Corporation: 404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Insulet_Corporation


,Company Name,Link to Company Wikipedia page,GICS Sector,GICS Sub-Industry,Headquarters Location,Industry,Revenue (Billions USD),Net Income (Billions USD),Number of Employees,Market Cap (Billions USD),CEO,Founder(s),Founded (year)
0,3M,https://en.wikipedia.org/wiki/3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",Conglomerate,32.68,6.99,85000,None,William M. Brown,"J. Danley Budd, Henry S. Bryan, William A. McG...",1902
1,A. O. Smith,https://en.wikipedia.org/wiki/A._O._Smith,Industrials,Building Products,"Milwaukee, Wisconsin",Water technology,2.9,0.37,15100,None,Kevin J. Wheeler,Charles Jeremiah Smith,1874
2,Abbott Laboratories,https://en.wikipedia.org/wiki/Abbott_Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois","Health care, Medical devices, Pharmaceutical",40.1,5.72,114000,None,Robert B. Ford,Wallace Calvin Abbott,1888
3,AbbVie,https://en.wikipedia.org/wiki/AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",Biopharmaceutical,54.32,4.873,50000,None,Richard A. Gonzalez,None,2012
4,Accenture,https://en.wikipedia.org/wiki/Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland","Professional services, Information technology ...",64.9,7.42,774000,None,Julie Sweet,None,1989


Finally, let's export the updated dataframe with additional columns to a CSV:

In [85]:
# Export to CSV
df.to_csv("sp500_detailed.csv", index=False)